# Beta Mixed Effects Models
Created: 08/08/24 \
Updated: //2024 \


In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.mixed_linear_model import MixedLM 
from joblib import Parallel, delayed
import pickle
import itertools
import time 
from matplotlib.ticker import StrMethodFormatter
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib as mpl
from matplotlib import cm



import warnings
warnings.filterwarnings('ignore')




In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Specify root directory for un-archived data and results 
base_dir   = '/sc/arion/projects/guLab/Alie/SWB/'
anat_dir   = f'{base_dir}ephys_analysis/recon_labels/'
neural_dir = f'{base_dir}ephys_analysis/data/'
behav_dir  = f'{base_dir}swb_behav_models/data/behavior_preprocessed/'
save_dir   = f'{base_dir}ephys_analysis/results/regression_data/'
script_dir = '/hpc/users/finka03/swb_ephys_analysis/scripts/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

08082024


In [4]:
import sys
sys.path.append(f'{base_dir}ephys_analysis/LFPAnalysis/')

from LFPAnalysis import analysis_utils

sys.path.append(f'{script_dir}analysis_notebooks/')

from ieeg_tools import *

In [5]:
subj_ids = list(pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)
# subj_ids


In [6]:
bdi_list = pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[3])
bdi_list = list(bdi_list.SWB_BDI)
subj_info_df = pd.DataFrame({'subj_id':subj_ids,'bdi':bdi_list})
subj_info_df



,subj_id,bdi
0,MS002,14
1,MS003,8
2,MS009,16
3,MS011,13
4,MS015,26
5,MS016,10
6,MS017,26
7,MS019,12
8,MS022,10
9,MS024,16


# Load Regression DF

In [7]:
band = 'beta'

In [8]:
band_df = pd.read_csv(f'{save_dir}{band}_master_df.csv')
band_df.subj_id.unique()

array(['MS002', 'MS003', 'MS009', 'MS011', 'MS015', 'MS016', 'MS017',
       'MS019', 'MS022', 'MS024', 'MS025', 'MS026', 'MS027', 'MS028',
       'MS029', 'MS030', 'MS033', 'MS035', 'MS041', 'MS043', 'MS048',
       'MS050', 'DA8', 'DA023', 'DA026', 'DA037', 'DA039'], dtype=object)

In [9]:
roi_reref_labels_master_df = pd.read_csv(glob(f'{base_dir}ephys_analysis/results/roi_info/roi_reref_labels_master.csv')[0])
roi_reref_labels_master_df = roi_reref_labels_master_df.drop(columns=['Unnamed: 0'])




In [10]:
roi_reref_labels_master_df.roi.value_counts()

dmpfc            216
ofc              202
acc              193
sts              155
hpc              135
amy              113
dlpfc            107
stg              105
vlpfc             82
ains              71
smg               66
pins              60
thalamus          58
mtg               57
temporal pole     31
mcc               28
motor             25
phg               25
parietal          24
vmpfc             23
fusiform          19
occipital         19
itg               15
angular           14
somat             11
premotor           9
caudate            8
pcc                1
Name: roi, dtype: int64

In [11]:
#### filter df by rois with electrode counts >= 50 *** updated to >15
keep_rois = roi_reref_labels_master_df.roi.value_counts().index[roi_reref_labels_master_df.roi.value_counts(
).values>=20].tolist()

# keep_rois = roi_reref_labels_master_df.roi.value_counts().index[
#     roi_reref_labels_master_df.roi.value_counts().values>=50].tolist()
# keep_rois = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy','mtg']     
keep_rois


['dmpfc',
 'ofc',
 'acc',
 'sts',
 'hpc',
 'amy',
 'dlpfc',
 'stg',
 'vlpfc',
 'ains',
 'smg',
 'pins',
 'thalamus',
 'mtg',
 'temporal pole',
 'mcc',
 'motor',
 'phg',
 'parietal',
 'vmpfc']

In [12]:
####### NEW DATA FRAME 
reg_df = band_df.copy()
reg_df = reg_df[reg_df.roi.isin(keep_rois)]
len(reg_df)

244136

In [13]:
np.max(reg_df.band_pow)

6.032583972049991

In [14]:
np.min(reg_df.band_pow)

-0.9714854334507784

# POW ~ Outcome Vars Regressions

In [15]:
outcome_var  = 'band_pow'
rand_eff_var = 'unique_reref_ch'

In [16]:
final_vars   = ['decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

               Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    band_pow  
No. Observations:      244112     Method:                ML        
No. Groups:            1776       Scale:                 0.0417    
Min. group size:       65         Log-Likelihood:        38019.1375
Max. group size:       148        Converged:             Yes       
Mean group size:       137.5                                       
-------------------------------------------------------------------
                        Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept               -0.055    0.002 -22.454 0.000 -0.060 -0.050
decisionCPE              0.006    0.001   5.955 0.000  0.004  0.007
Group Var                0.010    0.002                            
Group x decisionCPE Cov  0.000    0.000                            
decisionCPE Var          0.000    0.000                        

-75963.84280269922

In [17]:
final_vars   = ['RPE','decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

               Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    band_pow  
No. Observations:      244112     Method:                ML        
No. Groups:            1776       Scale:                 0.0416    
Min. group size:       65         Log-Likelihood:        38095.9012
Max. group size:       148        Converged:             Yes       
Mean group size:       137.5                                       
-------------------------------------------------------------------
                        Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept               -0.055    0.002 -22.449 0.000 -0.060 -0.050
RPE                      0.007    0.001   5.412 0.000  0.004  0.009
decisionCPE              0.001    0.001   0.719 0.472 -0.001  0.003
Group Var                0.010    0.002                            
Group x RPE Cov         -0.000    0.001                        

-76067.74855549095

In [18]:
final_vars   = ['TrialEV','RPE','decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

                Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     band_pow  
No. Observations:      244112      Method:                 ML        
No. Groups:            1776        Scale:                  0.0415    
Min. group size:       65          Log-Likelihood:         38159.6036
Max. group size:       148         Converged:              Yes       
Mean group size:       137.5                                         
---------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                 -0.055    0.002 -22.454 0.000 -0.060 -0.050
TrialEV                    0.005    0.001   5.644 0.000  0.003  0.007
RPE                        0.007    0.001   5.153 0.000  0.004  0.009
decisionCPE                0.001    0.001   0.931 0.352 -0.001  0.003
Group Var                  0.010    

-76133.12655751391

In [19]:
final_vars   = ['C(GambleChoice)', 'TotalProfit', 'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

                        Mixed Linear Model Regression Results
Model:                       MixedLM          Dependent Variable:          band_pow  
No. Observations:            244112           Method:                      ML        
No. Groups:                  1776             Scale:                       0.0408    
Min. group size:             65               Log-Likelihood:              39057.2397
Max. group size:             148              Converged:                   Yes       
Mean group size:             137.5                                                   
-------------------------------------------------------------------------------------
                                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------
Intercept                                 -0.053    0.003 -20.559 0.000 -0.058 -0.048
C(GambleChoice)[T.safe]                   -0.006    0.001  -4.528 0.000 -0.009

-77928.39857466736

In [20]:
final_vars   = ['C(TrialType)','TrialEV', 'TotalProfit', 'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic    

                          Mixed Linear Model Regression Results
Model:                        MixedLM            Dependent Variable:            band_pow  
No. Observations:             244112             Method:                        ML        
No. Groups:                   1776               Scale:                         0.0411    
Min. group size:              65                 Log-Likelihood:                38547.8147
Max. group size:              148                Converged:                     Yes       
Mean group size:              137.5                                                       
------------------------------------------------------------------------------------------
                                               Coef.  Std.Err.    z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------
Intercept                                      -0.054    0.003 -21.404 0.000 -0.059 -0.049
C(TrialType)[T.loss]      

-76748.27878718339

In [21]:
final_vars   = ['C(GambleChoice)', 'TrialEV','TotalProfit', 'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

                        Mixed Linear Model Regression Results
Model:                       MixedLM          Dependent Variable:          band_pow  
No. Observations:            244112           Method:                      ML        
No. Groups:                  1776             Scale:                       0.0408    
Min. group size:             65               Log-Likelihood:              39073.2571
Max. group size:             148              Converged:                   Yes       
Mean group size:             137.5                                                   
-------------------------------------------------------------------------------------
                                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------
Intercept                                 -0.053    0.003 -20.722 0.000 -0.058 -0.048
C(GambleChoice)[T.safe]                   -0.005    0.001  -3.626 0.000 -0.008

-77886.00117049165

In [22]:
final_vars   = ['C(GambleChoice)', 'TrialEV', 'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

                        Mixed Linear Model Regression Results
Model:                       MixedLM          Dependent Variable:          band_pow  
No. Observations:            244112           Method:                      ML        
No. Groups:                  1776             Scale:                       0.0412    
Min. group size:             65               Log-Likelihood:              38674.3712
Max. group size:             148              Converged:                   Yes       
Mean group size:             137.5                                                   
-------------------------------------------------------------------------------------
                                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------
Intercept                                 -0.053    0.003 -20.750 0.000 -0.059 -0.048
C(GambleChoice)[T.safe]                   -0.005    0.001  -3.362 0.001 -0.008

-77162.6616648732

In [23]:
final_vars   = ['TrialEV', 'TotalProfit', 'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      band_pow  
No. Observations:        244112       Method:                  ML        
No. Groups:              1776         Scale:                   0.0413    
Min. group size:         65           Log-Likelihood:          38465.1715
Max. group size:         148          Converged:               Yes       
Mean group size:         137.5                                           
-------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                     -0.055    0.002 -22.447 0.000 -0.060 -0.050
TrialEV                        0.005    0.001   5.810 0.000  0.004  0.007
TotalProfit                    0.005    0.001   4.226 0.000  0.003  0.008
decisionCPE                    0.005    0.001   5.704 0.

-76744.26232072695

In [24]:
final_vars   = ['C(GambleChoice)', 'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

                        Mixed Linear Model Regression Results
Model:                       MixedLM          Dependent Variable:          band_pow  
No. Observations:            244112           Method:                      ML        
No. Groups:                  1776             Scale:                       0.0412    
Min. group size:             65               Log-Likelihood:              38656.6641
Max. group size:             148              Converged:                   Yes       
Mean group size:             137.5                                                   
-------------------------------------------------------------------------------------
                                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------
Intercept                                 -0.053    0.003 -20.586 0.000 -0.058 -0.048
C(GambleChoice)[T.safe]                   -0.006    0.001  -4.273 0.000 -0.009

-77189.27434340885

In [25]:
final_vars   = ['TotalProfit', 'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      band_pow  
No. Observations:        244112       Method:                  ML        
No. Groups:              1776         Scale:                   0.0414    
Min. group size:         65           Log-Likelihood:          38398.0126
Max. group size:         148          Converged:               Yes       
Mean group size:         137.5                                           
-------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------
Intercept                     -0.055    0.002 -22.451 0.000 -0.060 -0.050
TotalProfit                    0.005    0.001   4.349 0.000  0.003  0.008
decisionCPE                    0.005    0.001   5.702 0.000  0.003  0.007
Group Var                      0.010    0.002           

-76671.9713105091

In [26]:
final_vars   = ['TrialEV', 'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic  

                Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     band_pow  
No. Observations:      244112      Method:                 ML        
No. Groups:            1776        Scale:                  0.0416    
Min. group size:       65          Log-Likelihood:         38085.5041
Max. group size:       148         Converged:              Yes       
Mean group size:       137.5                                         
---------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                 -0.055    0.002 -22.447 0.000 -0.060 -0.050
TrialEV                    0.005    0.001   5.768 0.000  0.004  0.007
decisionCPE                0.006    0.001   5.996 0.000  0.004  0.007
Group Var                  0.010    0.002                            
Group x TrialEV Cov       -0.000    

-76046.95446374474

In [27]:
final_vars   = ['C(TrialType)','decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic    

                          Mixed Linear Model Regression Results
Model:                        MixedLM            Dependent Variable:            band_pow  
No. Observations:             244112             Method:                        ML        
No. Groups:                   1776               Scale:                         0.0416    
Min. group size:              65                 Log-Likelihood:                38105.0610
Max. group size:              148                Converged:                     Yes       
Mean group size:              137.5                                                       
------------------------------------------------------------------------------------------
                                               Coef.  Std.Err.    z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------
Intercept                                      -0.052    0.003 -20.931 0.000 -0.057 -0.047
C(TrialType)[T.loss]      

-76024.04129696668

In [28]:
final_vars   = ['C(TrialType)','TotalProfit', 'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic    

                          Mixed Linear Model Regression Results
Model:                        MixedLM            Dependent Variable:            band_pow  
No. Observations:             244112             Method:                        ML        
No. Groups:                   1776               Scale:                         0.0412    
Min. group size:              65                 Log-Likelihood:                38485.8225
Max. group size:              148                Converged:                     Yes       
Mean group size:              137.5                                                       
------------------------------------------------------------------------------------------
                                               Coef.  Std.Err.    z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------
Intercept                                      -0.052    0.003 -20.960 0.000 -0.057 -0.048
C(TrialType)[T.loss]      

-76711.13190240585

In [29]:
final_vars   = ['C(TrialType)','TrialEV',  'decisionCPE']
final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
final_fit.bic    

                          Mixed Linear Model Regression Results
Model:                        MixedLM            Dependent Variable:            band_pow  
No. Observations:             244112             Method:                        ML        
No. Groups:                   1776               Scale:                         0.0415    
Min. group size:              65                 Log-Likelihood:                38163.6315
Max. group size:              148                Converged:                     Yes       
Mean group size:              137.5                                                       
------------------------------------------------------------------------------------------
                                               Coef.  Std.Err.    z    P>|z| [0.025 0.975]
------------------------------------------------------------------------------------------
Intercept                                      -0.054    0.003 -21.343 0.000 -0.059 -0.049
C(TrialType)[T.loss]      

-76066.75006845719


## ROI REGRESSIONS


In [ ]:
# test_rois = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins', 'acc','hpc','amy','motor']


In [30]:
outcome_var  = 'band_pow'
rand_eff_var = 'unique_reref_ch'
# final_vars   = ['C(GambleChoice)','TotalProfit','decisionCPE']
final_vars   = ['C(GambleChoice)','TrialEV','TotalProfit','decisionCPE']

In [31]:
roi_fit_dict = {f'{roi}':fit_mixed_model(reg_df[reg_df.roi==roi],
                                        final_vars,outcome_var,rand_eff_var,reml=False) 
                for roi in reg_df.roi.unique().tolist()}

roi_fit_results = []

for roi in list(roi_fit_dict.keys()):
    roi_fit = mixed_eff_results_df(roi_fit_dict[roi],reg_df[reg_df.roi==roi])
    roi_fit_results.append(roi_fit)
    
roi_fit_results = pd.concat(roi_fit_results).reset_index(drop=True)



In [32]:
roi_fit

,unique_reref_ch,subj_id,roi,bdi,raw_Intercept,raw_GambleChoice,raw_TrialEV,raw_TotalProfit,raw_decisionCPE,Intercept,GambleChoice,TrialEV,TotalProfit,decisionCPE
0,MS025_lcmip1-lcmip2,MS025,thalamus,10,-0.137837,0.026681,0.008997,0.025172,-0.019607,-0.154638,0.029018,0.025436,0.038606,-0.005585
1,MS025_lcmip2-lcmip3,MS025,thalamus,10,-0.114022,0.046799,0.025621,0.016698,-0.028217,-0.130823,0.049136,0.042060,0.030132,-0.014195
2,MS025_lcmip3-lcmip4,MS025,thalamus,10,-0.313089,0.011671,-0.007927,0.030804,-0.034458,-0.329890,0.014008,0.008512,0.044238,-0.020436
3,MS025_lcmip4-lcmip5,MS025,thalamus,10,-0.280519,0.044810,0.016697,0.031544,-0.043394,-0.297320,0.047146,0.033136,0.044978,-0.029372
4,MS025_lcmip5-lcmip6,MS025,thalamus,10,-0.212847,0.021787,0.004626,0.019862,-0.030083,-0.229648,0.024124,0.021065,0.033296,-0.016061
5,MS025_rcmip1-rcmip2,MS025,thalamus,10,-0.001441,0.044670,0.026415,0.015555,-0.012500,-0.018242,0.047007,0.042853,0.028989,0.001522
6,MS025_rcmip2-rcmip3,MS025,thalamus,10,-0.007344,0.036951,0.021031,0.014535,-0.010176,-0.024146,0.039287,0.037470,0.027970,0.003846
7,MS025_rcmip3-rcmip4,MS025,thalamus,10,-0.002218,-0.019567,-0.011001,-0.011649,0.003126,-0.019019,-0.017230,0.005437,0.001785,0.017149
8,MS025_rcmip4-rcmip5,MS025,thalamus,10,-0.027302,0.023606,0.017928,-0.006557,-0.016581,-0.044103,0.025942,0.034367,0.006877,-0.002559
9,MS025_rcmip5-rcmip6,MS025,thalamus,10,0.003284,-0.015760,-0.004479,-0.026826,-0.004093,-0.013518,-0.013423,0.011960,-0.013392,0.009929


In [33]:
roi_fit_results

,unique_reref_ch,subj_id,roi,bdi,raw_Intercept,raw_GambleChoice,raw_TrialEV,raw_TotalProfit,raw_decisionCPE,Intercept,GambleChoice,TrialEV,TotalProfit,decisionCPE
0,DA023_lac1-lac2,DA023,acc,14,0.008821,-0.019932,0.002223,-0.011230,0.007151,0.003642,-0.032425,0.005496,-0.010194,0.014319
1,DA023_lac2-lac3,DA023,acc,14,0.012320,-0.004611,-0.006981,0.013845,-0.005091,0.007140,-0.017104,-0.003708,0.014881,0.002077
2,DA023_lac3-lac4,DA023,acc,14,0.009223,-0.002647,-0.003692,0.004785,-0.002689,0.004043,-0.015140,-0.000419,0.005821,0.004479
3,MS002_lacas1-lacas2,MS002,acc,14,0.025398,-0.023713,0.008948,0.013541,0.014771,0.020218,-0.036206,0.012221,0.014577,0.021939
4,MS002_lacas2-lacas3,MS002,acc,14,0.002349,0.013979,0.001369,0.014037,0.008953,-0.002831,0.001486,0.004641,0.015073,0.016121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771,MS050_lcmang3-lcmang4,MS050,thalamus,9,0.011630,-0.007046,-0.002037,-0.005841,0.002695,-0.005172,-0.004710,0.014402,0.007593,0.016718
1772,MS050_lcmang4-lcmang5,MS050,thalamus,9,0.017397,0.010288,0.012430,-0.016949,-0.008583,0.000595,0.012625,0.028869,-0.003514,0.005439
1773,MS050_rcmsmg1-rcmsmg2,MS050,thalamus,9,0.011963,0.013797,0.009607,-0.001646,-0.005015,-0.004839,0.016134,0.026045,0.011788,0.009008
1774,MS050_rcmsmg2-rcmsmg3,MS050,thalamus,9,0.009769,-0.017888,-0.012898,0.001505,0.009046,-0.007032,-0.015551,0.003540,0.014939,0.023068


In [34]:
fe_params_results = {f'{roi}':roi_fit_dict[roi].fe_params for roi in list(roi_fit_dict.keys())}
# fe_params_pvals = {f'{roi}':roi_fit_dict[roi].pvalues for roi in list(roi_fit_dict.keys())}


In [35]:
# fe_params_results

In [36]:
fe_results = []
for fe_res in fe_params_results:
    
    fe_results.append(pd.DataFrame({**dict(fe_params_results[fe_res]),**{'roi':fe_res}},index=[0]))


In [37]:
fe_results = pd.concat(fe_results).reset_index(drop=True)

In [38]:
fe_results['roi_cpe_pval'] = [np.round(roi_fit_dict[roi].pvalues['decisionCPE'],4) for roi in fe_results.roi.tolist()]

In [39]:
fe_results

,Intercept,C(GambleChoice)[T.safe],TrialEV,TotalProfit,decisionCPE,roi,roi_cpe_pval
0,-0.005180,-0.012493,0.003273,0.001036,0.007168,acc,0.0012
1,-0.035581,-0.008979,-0.003294,0.010663,0.007822,dmpfc,0.0029
2,-0.090577,0.009229,0.001078,0.011191,0.002202,amy,0.5439
3,-0.070407,-0.026212,0.010891,0.009746,-0.005244,sts,0.1692
4,0.020233,-0.015719,0.000784,0.009796,0.025027,ains,0.0000
5,-0.029090,-0.013844,-0.000858,0.005622,0.013278,vlpfc,0.0019
6,-0.122107,0.009181,0.004811,-0.005928,0.001439,hpc,0.7151
7,-0.096133,-0.025194,0.002145,0.007144,-0.013647,mtg,0.0265
8,-0.042171,0.001608,0.002816,0.004540,0.006796,ofc,0.0081
9,-0.059945,0.007357,0.010741,0.001899,0.019369,pins,0.0003


In [40]:
fe_results

,Intercept,C(GambleChoice)[T.safe],TrialEV,TotalProfit,decisionCPE,roi,roi_cpe_pval
0,-0.005180,-0.012493,0.003273,0.001036,0.007168,acc,0.0012
1,-0.035581,-0.008979,-0.003294,0.010663,0.007822,dmpfc,0.0029
2,-0.090577,0.009229,0.001078,0.011191,0.002202,amy,0.5439
3,-0.070407,-0.026212,0.010891,0.009746,-0.005244,sts,0.1692
4,0.020233,-0.015719,0.000784,0.009796,0.025027,ains,0.0000
5,-0.029090,-0.013844,-0.000858,0.005622,0.013278,vlpfc,0.0019
6,-0.122107,0.009181,0.004811,-0.005928,0.001439,hpc,0.7151
7,-0.096133,-0.025194,0.002145,0.007144,-0.013647,mtg,0.0265
8,-0.042171,0.001608,0.002816,0.004540,0.006796,ofc,0.0081
9,-0.059945,0.007357,0.010741,0.001899,0.019369,pins,0.0003


In [54]:

# sns.set_context("talk",rc={'axes.linewidth': 1.5})

# for roi in roi_plot_data.roi.unique().tolist()[:-1]:
#     roi_temp_data = roi_plot_data[roi_plot_data.roi==roi]

#     rho   = rho_data[rho_data.roi == roi].round_rho.values[0]
#     rho_p = rho_data[rho_data.roi == roi].round_p.values[0]
    
        
#     fig,ax = plt.subplots(1,1,figsize=(5,3.5),dpi=300)

#     sns.regplot(data=roi_temp_data,x='bdi',y='elec_bCPE',scatter=False,ci=95,
#                 fit_reg=True,color = '#00467e') #034975
    
#     sns.scatterplot(data = roi_temp_data.groupby(['subj_id']).agg('mean'),x='bdi',
#                     y='raw_decisionCPE',color = '#00467e',edgecolor=None,alpha=0.8)
    
# #     plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 
#     plt.xlabel('BDI-II')
# #     plt.xlim([0,40])
    
#     plt.tick_params(length=0) 
# #     plt.ylabel('beta'+r'$beta_{{{zpow}}}$'+' cpe encoding')
#     plt.ylabel(fr'${band}_{{{zpow}}} \sim cpe$')

#     plt.text(np.max(roi_temp_data.bdi)-8,
#              np.round(np.max(roi_temp_data.groupby(['subj_id']).agg('mean').elec_bCPE),3)-0.003,
#              r'$rho$ = '+f'{rho} \n'+ fr'$p \approxeq $ {rho_p}',color='k',fontsize=12)
# #              color='#00467e',fontsize=16)
# #     plt.text(0.8,0.8,r'$rho$ = '+f'{rho} \n'+ fr'$p \approxeq $ {rho_p}',
# #             verticalalignment='top', horizontalalignment='right',
# #             color='k',fontsize=16)

#     plt.title(f'{roi}',fontsize=20)
#     plt.tight_layout()


    
#     sns.despine()
    
# #     plt.savefig(f'{fig_save_dir}/{roi}_{rho}_{rho_p}_roi_bdi_beta-cpe-slope.pdf', format='pdf', metadata=None,
# #         bbox_inches='tight', pad_inches=0.1,dpi=300,
# #         facecolor='auto', edgecolor='auto',
# #         backend=None)
    

In [ ]:
sig_roi_fit_results

In [ ]:
roi_fit_results

In [ ]:
bonferr_alpha = 0.05/len(test_rois)

In [ ]:
bonferr_alpha

In [ ]:
fe_results.roi[fe_results.roi_cpe_pval <= bonferr_alpha]

In [ ]:
sig_roi_fit_results = roi_fit_results[roi_fit_results.decisionCPE <= 0.05]

In [ ]:
sig_roi_fit_results.roi.value_counts()

In [ ]:
sig_roi_fit_results

In [ ]:
plot_roi_model_results(sig_data,'roi','Z_Score_decisionCPE',bar_order=bar_order,color='#135546',figsize=(10,4))

In [ ]:
final_fit.pvalues['decisionCPE']

In [ ]:
# final_fit.t_test_pairwise('C(TrialType)',
#                           factor_labels=[ 'TrialEV', 'TotalProfit', 'decisionCPE'])
# final_fit.model.data.design_info.term_names.index(term_name)

In [ ]:
mixed_effects_ftest_ttest(final_fit)

In [ ]:
fe_param_names = list(final_fit.fe_params.index[1:])
fe_param_names

In [ ]:
plot_data = pd.concat([pd.DataFrame(
    {**{'subj_id':elec_id.split('_')[0],'reref_ch_names':elec_id.split('_')[1]},'unique_reref_ch':elec_id,
     **{f'raw_{param}':final_fit.random_effects[elec_id][param] for param in list(
            final_fit.random_effects[elec_id].index)}},index=[range(len(fe_param_names))])
        
    for elec_id in final_fit.random_effects.keys()]).reset_index(drop=True)

plot_data['roi'] = [reg_df.roi[reg_df.unique_reref_ch == row.unique_reref_ch].values[0]
                                for idx,row in plot_data.iterrows()]

plot_data['bdi'] = [subj_info_df.bdi[subj_info_df.subj_id == subj_id].values[0]
                                for subj_id in plot_data.subj_id]

In [ ]:
plot_data

In [ ]:
#### Get all results for Figs!!!!

fe_intercept   = final_fit.params['Intercept']
fe_cpe_beta    = final_fit.params['decisionCPE']
fe_cpe_beta


In [ ]:
#### add fixed effects data to all_elec_plot_data
plot_data['elec_b0']   = np.add(plot_data.raw_Group,fe_intercept)
plot_data['elec_bCPE'] = np.add(plot_data.raw_decisionCPE,fe_cpe_beta)
plot_data = plot_data[~plot_data.roi.isnull()]

In [ ]:
plot_data

In [ ]:
raw_roi_stats = {f'{roi}':ttest_1samp(plot_data.raw_decisionCPE[plot_data.roi == roi].unique(),
                 popmean=0) for roi in plot_data.roi.unique().tolist()}
raw_roi_stats

In [ ]:
roi_stats = {f'{roi}':ttest_1samp(plot_data.elec_bCPE[plot_data.roi == roi].unique(),
                 popmean=0) for roi in plot_data.roi.unique().tolist()}
roi_stats

In [ ]:
pairwise_ains_vals = plot_data.elec_bCPE[plot_data.roi == 'ains'].unique()

pairwise_roi_stats = {f'ains={roi}':ttest_ind(pairwise_ains_vals,plot_data.elec_bCPE[plot_data.roi == roi].unique(),
                 ) for roi in plot_data.roi.unique().tolist()}
pairwise_roi_stats


In [ ]:
pairwise_ains_vals = plot_data.elec_bCPE[plot_data.roi == 'ains'].unique()

pairwise_roi_stats_greater = {f'ains={roi}':ttest_ind(pairwise_ains_vals,plot_data.elec_bCPE[plot_data.roi == roi].unique(),
                 alternative='greater') for roi in plot_data.roi.unique().tolist()}
pairwise_roi_stats_greater

In [ ]:
pairwise_ains_vals = plot_data.elec_bCPE[plot_data.roi == 'ains'].unique()

pairwise_roi_stats_less = {f'ains={roi}':ttest_ind(pairwise_ains_vals,plot_data.elec_bCPE[plot_data.roi == roi].unique(),
                 alternative='less') for roi in plot_data.roi.unique().tolist()}
pairwise_roi_stats_less

In [ ]:
bonferr_alpha = 0.05/(len(plot_data.roi.unique().tolist()))
bonferr_alpha

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,raw_roi_stats[roi].pvalue) for roi in raw_roi_stats.keys()
                                                   if raw_roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())

sig_raw = [(roi,raw_roi_stats[roi].pvalue) for roi in raw_roi_stats.keys()
                                                   if raw_roi_stats[roi].pvalue< bonferr_alpha]

sig_raw_rois = list(zip(*sig_raw))[0]

In [ ]:
sig_raw_rois

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,roi_stats[roi].pvalue) for roi in roi_stats.keys()
                                                   if roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())
sig_bcpe = [(roi,roi_stats[roi].pvalue) for roi in roi_stats.keys()
                                                   if roi_stats[roi].pvalue< bonferr_alpha]

sig_bcpe_rois = list(zip(*sig_bcpe))[0]

In [ ]:
list(zip(*sig_bcpe))[0]

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,pairwise_roi_stats_greater[roi].pvalue) for 
                                                   roi in pairwise_roi_stats_greater.keys()
                                                   if pairwise_roi_stats_greater[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())
sig_bcpe = [(roi,pairwise_roi_stats_greater[roi].pvalue) for roi in pairwise_roi_stats_greater.keys()
                                                   if pairwise_roi_stats_greater[roi].pvalue< bonferr_alpha]

sig_bcpe_rois = list(zip(*sig_bcpe))[0]

In [ ]:
sig_bcpe_rois 

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,pairwise_roi_stats_less[roi].pvalue) for 
                                                   roi in pairwise_roi_stats_less.keys()
                                                   if pairwise_roi_stats_less[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())
# sig_bcpe = [(roi,pairwise_roi_stats_less[roi].pvalue) for roi in pairwise_roi_stats_less.keys()
#                                                    if pairwise_roi_stats_less[roi].pvalue< bonferr_alpha]

# sig_bcpe_rois = list(zip(*sig_bcpe))[0]

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,pairwise_roi_stats[roi].pvalue) for 
                                                   roi in pairwise_roi_stats.keys()
                                                   if pairwise_roi_stats[roi].pvalue< bonferr_alpha])

sig_bcpe = [(roi,pairwise_roi_stats[roi].pvalue) for roi in pairwise_roi_stats.keys()
                                                   if pairwise_roi_stats[roi].pvalue< bonferr_alpha]

sig_bcpe_rois = list(zip(*sig_bcpe))[0]

In [ ]:
sig_bcpe_rois

In [ ]:
len([(roi,roi_stats[roi].pvalue) for roi in roi_stats.keys()
                                                   if roi_stats[roi].pvalue< bonferr_alpha])

In [ ]:
roi_stats.keys()

In [ ]:
# roi_order = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy','mtg','motor','thalamus']
roi_order = ['dlpfc','vlpfc','ofc','ains','pins','acc','hpc','amy']

In [ ]:
roi_plot_data = plot_data.copy()
roi_plot_data = roi_plot_data[roi_plot_data.roi.isin(roi_order)]

# roi_plot_names = {'thalamus':'THAL','hpc':'HPC','amy':'AMY','acc':'ACC','ofc':'OFC','ains':'aINS',
#                   'pins':'pINS','dmpfc':'dmPFC','dlpfc':'dlPFC','vlpfc':'vlPFC'}


In [ ]:
sns.set_context("poster",rc={'axes.linewidth': 1.5})
fig,ax = plt.subplots(figsize=(10, 4),dpi=500)

fig_save_dir = f'{base_dir}ephys_analysis/figs/beta_regressions/'
os.makedirs(fig_save_dir,exist_ok=True)

sns.barplot(data = roi_plot_data,
            x = 'roi',
            y = 'elec_bCPE',
#             y='raw_decisionCPE',
#             color = '#561C46',
#             color = '#00467e',
#             color = '#346a97',
            color = '#346a97',
            saturation=1,
            order=roi_order,
            errorbar=None) 


sns.stripplot(data = roi_plot_data,
            x = 'roi',
            y = 'elec_bCPE',
#               y='raw_decisionCPE',
#               color='#78496b', 
              color = '#00467e',
              alpha=.4,
              dodge=True,
              order=roi_order,
              size = 6)

ax.axhline(y=0,color='#00467e',linewidth=2)
plt.tick_params(length=0 )#,grid_linewidth=1)

plt.ylim([-0.05,0.05])
ax.set(xlabel=None)
# plt.ylabel('CPE Encoding in Beta Power (z)')
# plt.ylabel('beta(z) cpe encoding')
# plt.ylabel(r'$beta_{{{zpow}}}$'+ ' cpe encoding')
# plt.ylabel(r'$beta_{{{zpow}}}$'+ ' cpe encoding')
plt.ylabel(r'$beta_{{{zpow}}} \sim cpe$')
# plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}')) 
# plt.locator_params(axis='y', nbins=5,tight=True)


sns.despine()
    
plt.savefig(f'{fig_save_dir}/Bpow_CPE_barplot_{date}.pdf', format='pdf', metadata=None,
bbox_inches=None, pad_inches=0.1,dpi=500,
facecolor='auto', edgecolor='auto',
backend=None)


In [ ]:
roi_plot_data.roi.unique().tolist()

In [ ]:
bonferr_alpha

In [ ]:
rho_data = []
sig_rho = []

# bonferr_alpha = 0.05/len(roi_plot_data.roi.unique().tolist())
# bonferr_alpha = 0.05/9
bonferr_alpha = 0.01

# BDI CORRELATIONS
for roi in roi_plot_data.roi.unique().tolist()[:]:
    roi_temp_data = roi_plot_data[roi_plot_data.roi==roi]
    rho,rho_p     = spearmanr(roi_temp_data.bdi,roi_temp_data.elec_bCPE)
#     rho,rho_p     = pearsonr(roi_temp_data.bdi,roi_temp_data.elec_bCPE)
#     print(f'{roi} pearsonr',rho,rho_p)
    rho_data.append(pd.DataFrame({'roi':roi,'rho':rho,'rho_p':rho_p},index=[0]))
#     if rho_p <= bonferr_alpha:
#         sig_bdi_rois.append(roi)
#     if rho_p < bonferr_alpha:
    if rho_p < 0.05:
        sig_rho.append((roi,rho,rho_p))
    print(f'{roi} pearsonr',rho,rho_p)
rho_data = pd.concat(rho_data).reset_index(drop=True)


In [ ]:
rho_data

In [ ]:
sig_rho

In [ ]:
rho_data['round_rho'] = np.round(rho_data.rho,2)
rho_data['round_p']   = np.round(rho_data.rho_p,2)
rho_data['round_p'][rho_data.roi == 'acc'] = 0.01
rho_data['round_p'][rho_data.roi == 'ofc'] = 0.01
rho_data

In [ ]:

sns.set_context("talk",rc={'axes.linewidth': 1.5})

for roi in roi_plot_data.roi.unique().tolist()[:-1]:
    roi_temp_data = roi_plot_data[roi_plot_data.roi==roi]

    rho   = rho_data[rho_data.roi == roi].round_rho.values[0]
    rho_p = rho_data[rho_data.roi == roi].round_p.values[0]
    
        
    fig,ax = plt.subplots(1,1,figsize=(5,3.5),dpi=300)

    sns.regplot(data=roi_temp_data,x='bdi',y='elec_bCPE',scatter=False,ci=95,
                fit_reg=True,color = '#00467e') #034975
    
    sns.scatterplot(data = roi_temp_data.groupby(['subj_id']).agg('mean'),x='bdi',
                    y='raw_decisionCPE',color = '#00467e',edgecolor=None,alpha=0.8)
    
#     plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 
    plt.xlabel('BDI-II')
#     plt.xlim([0,40])
    
    plt.tick_params(length=0) 
#     plt.ylabel('beta'+r'$beta_{{{zpow}}}$'+' cpe encoding')
    plt.ylabel(r'$beta_{{{zpow}}} \sim cpe$')

    plt.text(np.max(roi_temp_data.bdi)-8,
             np.round(np.max(roi_temp_data.groupby(['subj_id']).agg('mean').elec_bCPE),3)-0.003,
             r'$rho$ = '+f'{rho} \n'+ fr'$p \approxeq $ {rho_p}',color='k',fontsize=12)
#              color='#00467e',fontsize=16)
#     plt.text(0.8,0.8,r'$rho$ = '+f'{rho} \n'+ fr'$p \approxeq $ {rho_p}',
#             verticalalignment='top', horizontalalignment='right',
#             color='k',fontsize=16)

    plt.title(f'{roi}',fontsize=20)
    plt.tight_layout()


    
    sns.despine()
    
    plt.savefig(f'{fig_save_dir}/{roi}_{rho}_{rho_p}_roi_bdi_beta-cpe-slope.pdf', format='pdf', metadata=None,
        bbox_inches='tight', pad_inches=0.1,dpi=300,
        facecolor='auto', edgecolor='auto',
        backend=None)
    

# Band x BDI Interaction
- Rerun final mixed effects model on each ROI separately
- Correlate random effects slopes for each regressor with BDI

In [ ]:
rand_eff_var = 'unique_reref_ch'


In [ ]:
reg_df

In [ ]:
bdi_vars    = ['decisionCPE','bdi','decisionCPE:bdi']
outcome_var = 'band_pow'

bdi_fit = fit_mixed_model(reg_df,bdi_vars,outcome_var,rand_eff_var,reml=False)
print(bdi_fit.summary())
    

In [ ]:
fe_param_names = list(bdi_fit.fe_params.index[1:])
fe_param_names

In [ ]:
bdi_plot_data = pd.concat([pd.DataFrame(
    {**{'subj_id':elec_id.split('_')[0],'reref_ch_names':elec_id.split('_')[1]},'unique_reref_ch':elec_id,
     **{f'raw_{param}':bdi_fit.random_effects[elec_id][param] for param in list(
            bdi_fit.random_effects[elec_id].index)}},index=[range(len(fe_param_names))])
        
    for elec_id in bdi_fit.random_effects.keys()]).reset_index(drop=True)

bdi_plot_data['roi'] = [reg_df.roi[reg_df.unique_reref_ch == row.unique_reref_ch].values[0]
                                for idx,row in bdi_plot_data.iterrows()]

bdi_plot_data['bdi'] = [subj_info_df.bdi[subj_info_df.subj_id == subj_id].values[0]
                                for subj_id in bdi_plot_data.subj_id]

In [ ]:
bdi_plot_data

In [ ]:
#### Get all results for Figs!!!!

fe_intercept   = bdi_fit.params['Intercept']
fe_cpe_beta    = bdi_fit.params['decisionCPE:bdi']
fe_cpe_beta


In [ ]:
#### add fixed effects data to all_elec_plot_data
bdi_plot_data['elec_b0']   = np.add(bdi_plot_data.raw_Group,fe_intercept)
bdi_plot_data['elec_bCPE'] = np.add(bdi_plot_data['raw_decisionCPE:bdi'],fe_cpe_beta)
bdi_plot_data = bdi_plot_data[~bdi_plot_data.roi.isnull()]

In [ ]:
bdi_plot_data

In [ ]:
# raw_bdi

raw_roi_stats = {f'{roi}':ttest_1samp(bdi_plot_data['raw_bdi'][bdi_plot_data.roi == roi].unique(),
                 popmean=0) for roi in bdi_plot_data.roi.unique().tolist()}
raw_roi_stats

In [ ]:
raw_roi_stats = {f'{roi}':ttest_1samp(bdi_plot_data['raw_decisionCPE:bdi'][bdi_plot_data.roi == roi].unique(),
                 popmean=0) for roi in bdi_plot_data.roi.unique().tolist()}
raw_roi_stats

In [ ]:
roi_stats = {f'{roi}':ttest_1samp(bdi_plot_data.elec_bCPE[bdi_plot_data.roi == roi].unique(),
                 popmean=0) for roi in bdi_plot_data.roi.unique().tolist()}
roi_stats

In [ ]:
pairwise_ains_vals = bdi_plot_data.elec_bCPE[plot_data.roi == 'ains'].unique()

pairwise_roi_stats = {f'ains={roi}':ttest_ind(pairwise_ains_vals,bdi_plot_data.elec_bCPE[bdi_plot_data.roi == roi].unique(),
                 ) for roi in bdi_plot_data.roi.unique().tolist()}
pairwise_roi_stats


In [ ]:
pairwise_ains_vals = bdi_plot_data.elec_bCPE[plot_data.roi == 'ains'].unique()

pairwise_roi_stats_greater = {f'ains={roi}':ttest_ind(pairwise_ains_vals,bdi_plot_data.elec_bCPE[bdi_plot_data.roi == roi].unique(),
                 alternative='greater') for roi in bdi_plot_data.roi.unique().tolist()}
pairwise_roi_stats_greater

In [ ]:
pairwise_ains_vals = bdi_plot_data.elec_bCPE[bdi_plot_data.roi == 'ains'].unique()

pairwise_roi_stats_less = {f'ains={roi}':ttest_ind(pairwise_ains_vals,bdi_plot_data.elec_bCPE[bdi_plot_data.roi == roi].unique(),
                 alternative='less') for roi in bdi_plot_data.roi.unique().tolist()}
pairwise_roi_stats_less

In [ ]:
bonferr_alpha = 0.05/len(roi_order)
bonferr_alpha

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,raw_roi_stats[roi].pvalue) for roi in raw_roi_stats.keys()
                                                   if raw_roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())

sig_raw = [(roi,raw_roi_stats[roi].pvalue) for roi in raw_roi_stats.keys()
                                                   if raw_roi_stats[roi].pvalue< bonferr_alpha]

sig_raw_rois = list(zip(*sig_raw))[0]

In [ ]:
sig_raw_rois

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,roi_stats[roi].pvalue) for roi in roi_stats.keys()
                                                   if roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())
sig_bcpe = [(roi,roi_stats[roi].pvalue) for roi in roi_stats.keys()
                                                   if roi_stats[roi].pvalue< bonferr_alpha]

sig_bcpe_rois = list(zip(*sig_bcpe))[0]

In [ ]:
list(zip(*sig_bcpe))[0]

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,pairwise_roi_stats_greater[roi].pvalue) for 
                                                   roi in pairwise_roi_stats_greater.keys()
                                                   if pairwise_roi_stats_greater[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())
sig_bcpe = [(roi,pairwise_roi_stats_greater[roi].pvalue) for roi in pairwise_roi_stats_greater.keys()
                                                   if pairwise_roi_stats_greater[roi].pvalue< bonferr_alpha]

# sig_bcpe_rois = list(zip(*sig_bcpe))[0]

In [ ]:
# sig_bcpe_rois 

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,pairwise_roi_stats_less[roi].pvalue) for 
                                                   roi in pairwise_roi_stats_less.keys()
                                                   if pairwise_roi_stats_less[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())
sig_bcpe = [(roi,pairwise_roi_stats_less[roi].pvalue) for roi in pairwise_roi_stats_less.keys()
                                                   if pairwise_roi_stats_less[roi].pvalue< bonferr_alpha]

sig_bcpe_rois = list(zip(*sig_bcpe))[0]

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,pairwise_roi_stats[roi].pvalue) for 
                                                   roi in pairwise_roi_stats.keys()
                                                   if pairwise_roi_stats[roi].pvalue< bonferr_alpha])

sig_bcpe = [(roi,pairwise_roi_stats[roi].pvalue) for roi in pairwise_roi_stats.keys()
                                                   if pairwise_roi_stats[roi].pvalue< bonferr_alpha]

sig_bcpe_rois = list(zip(*sig_bcpe))[0]

In [ ]:
sig_bcpe_rois

In [ ]:
len([(roi,roi_stats[roi].pvalue) for roi in roi_stats.keys()
                                                   if roi_stats[roi].pvalue< bonferr_alpha])

In [ ]:
roi_stats.keys()

In [ ]:
# roi_order = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy','mtg','motor','thalamus']
roi_order = ['dlpfc','vlpfc','ofc','ains','pins','acc','hpc','amy']

In [ ]:
roi_plot_data = bdi_plot_data.copy()
roi_plot_data = roi_plot_data[roi_plot_data.roi.isin(roi_order)]

# roi_plot_names = {'thalamus':'THAL','hpc':'HPC','amy':'AMY','acc':'ACC','ofc':'OFC','ains':'aINS',
#                   'pins':'pINS','dmpfc':'dmPFC','dlpfc':'dlPFC','vlpfc':'vlPFC'}


In [ ]:
bdi_plot_data.roi.unique()

In [ ]:
sns.set_context("poster",rc={'axes.linewidth': 1.5})
fig,ax = plt.subplots(figsize=(10, 4),dpi=500)

fig_save_dir = f'{base_dir}ephys_analysis/figs/{band}_regressions/'
os.makedirs(fig_save_dir,exist_ok=True)

sns.barplot(data = bdi_plot_data,
            x = 'roi',
            y = 'elec_bCPE',
#             y='raw_decisionCPE',
#             color = '#561C46',
#             color = '#00467e',
#             color = '#346a97',
            color = '#173b6f',
            saturation=1,
            order=roi_order,
            errorbar=None) 


sns.stripplot(data = bdi_plot_data,
            x = 'roi',
            y = 'elec_bCPE',
#               y='raw_decisionCPE',
#               color='#78496b', 
              color = '#173b6f',
              alpha=.4,
              dodge=True,
              order=roi_order,
              size = 6)

ax.axhline(y=0,color='#173b6f',linewidth=2)
plt.tick_params(length=0 )#,grid_linewidth=1)

# plt.ylim([-0.05,0.05])
ax.set(xlabel=None)
# plt.ylabel('CPE Encoding in Beta Power (z)')
# plt.ylabel('beta(z) cpe encoding')
# plt.ylabel(r'$beta_{{{zpow}}}$'+ ' cpe encoding')
# plt.ylabel(r'$beta_{{{zpow}}}$'+ ' cpe encoding')
plt.ylabel(fr'${band}_{{{zpow}}} \sim cpe:bdi$')
# plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}')) 
# plt.locator_params(axis='y', nbins=5,tight=True)


sns.despine()
    
plt.savefig(f'{fig_save_dir}/Bpow_CPE:BDI_barplot_{date}.pdf', format='pdf', metadata=None,
bbox_inches=None, pad_inches=0.1,dpi=500,
facecolor='auto', edgecolor='auto',
backend=None)


In [ ]:
bdi_plot_data.roi.unique().tolist()

In [ ]:
bonferr_alpha

In [ ]:
rho_data = []
sig_rho = []

# bonferr_alpha = 0.05/len(roi_plot_data.roi.unique().tolist())
# bonferr_alpha = 0.05/9
bonferr_alpha = 0.01

# BDI CORRELATIONS
for roi in roi_plot_data.roi.unique().tolist()[:]:
    roi_temp_data = roi_plot_data[roi_plot_data.roi==roi]
    rho,rho_p     = spearmanr(roi_temp_data.bdi,roi_temp_data.elec_bCPE)
#     rho,rho_p     = pearsonr(roi_temp_data.bdi,roi_temp_data.elec_bCPE)
#     print(f'{roi} pearsonr',rho,rho_p)
    rho_data.append(pd.DataFrame({'roi':roi,'rho':rho,'rho_p':rho_p},index=[0]))
#     if rho_p <= bonferr_alpha:
#         sig_bdi_rois.append(roi)
#     if rho_p < bonferr_alpha:
    if rho_p < 0.05:
        sig_rho.append((roi,rho,rho_p))
    print(f'{roi} pearsonr',rho,rho_p)
rho_data = pd.concat(rho_data).reset_index(drop=True)


In [ ]:
rho_data

In [ ]:
sig_rho

In [ ]:
rho_data['round_rho'] = np.round(rho_data.rho,2)
rho_data['round_p']   = np.round(rho_data.rho_p,3)
# rho_data['round_p'][rho_data.roi == 'acc'] = 0.01
# rho_data['round_p'][rho_data.roi == 'ofc'] = 0.01
rho_data

In [ ]:

sns.set_context("talk",rc={'axes.linewidth': 1.5})

for roi in roi_plot_data.roi.unique().tolist()[:-1]:
    roi_temp_data = roi_plot_data[roi_plot_data.roi==roi]

    rho   = rho_data[rho_data.roi == roi].round_rho.values[0]
    rho_p = rho_data[rho_data.roi == roi].round_p.values[0]
    
        
    fig,ax = plt.subplots(1,1,figsize=(5,3.5),dpi=300)

    sns.regplot(data=roi_temp_data,x='bdi',y='raw_decisionCPE',scatter=False,ci=95,
                fit_reg=True,color = '#173b6f') #034975
    
    sns.scatterplot(data = roi_temp_data.groupby(['subj_id']).agg('mean'),x='bdi',
                    y='raw_decisionCPE',color = '#173b6f',edgecolor=None,alpha=0.8)
    
#     plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 
    plt.xlabel('BDI-II')
#     plt.xlim([0,40])
    
    plt.tick_params(length=0) 
#     plt.ylabel('beta'+r'$beta_{{{zpow}}}$'+' cpe encoding')
    plt.ylabel(fr'${band}_{{{zpow}}} \sim cpe:bdi$')

    plt.text(np.max(roi_temp_data.bdi)-8,
             np.round(np.max(roi_temp_data.groupby(['subj_id']).agg('mean').raw_decisionCPE),3)-0.003,
             r'$rho$ = '+f'{rho} \n'+ fr'$p \approxeq $ {rho_p}',color='k',fontsize=12)
#              color='#00467e',fontsize=16)
#     plt.text(0.8,0.8,r'$rho$ = '+f'{rho} \n'+ fr'$p \approxeq $ {rho_p}',
#             verticalalignment='top', horizontalalignment='right',
#             color='k',fontsize=16)

    plt.title(f'{roi}',fontsize=20)
    plt.tight_layout()
    


    
    sns.despine()
    
    plt.savefig(f'{fig_save_dir}/{roi}_{rho}_{rho_p}_roi_bdi_{band}-cpe:bdi-slope.pdf', format='pdf', metadata=None,
        bbox_inches='tight', pad_inches=0.1,dpi=300,
        facecolor='auto', edgecolor='auto',
        backend=None)
    

In [ ]:
reg_df

In [ ]:
reg_df['Depressed'] = ['High BDI' if bdi >=20 else 'Low BDI' for bdi in reg_df.bdi.tolist()]

In [ ]:
## X plot attempt 

sns.set_theme(style='ticks') 
sns.set_context("poster")

# from matplotlib import rc
# rc('text', usetex=False)


fig,ax = plt.subplots(figsize=(8,5))
# sns.set_context("poster",rc={'axes.linewidth': 2})
# sns.set_theme(style='ticks') 

# palette={'depression':"#3f2755",'no depression':"#054a6f"}

# palette={'high BDI':"#0b4e5f",'low BDI':"#824973"}


# ax = sns.lmplot(data=reg_df,x='decisionCPE',y='band_pow',hue='Depression',scatter=False,ci=95,palette=palette)
# sns.move_legend(ax, loc='lower center', bbox_to_anchor=(0.1, 1),bbox_to_anchor=(-1, 0), ncol=2)

# sns.move_legend(ax,"upper center", bbox_to_anchor=(0.5, 1), borderaxespad=0.1,frameon=False,framealpha=0,
#                 fontsize='x-small',shadow=None,title=None,columnspacing=0.5,edgecolor=None,markerfirst=True,
#                 markerscale=4,labelcolor=None, ncol=2)


sns.regplot(data=reg_df[reg_df.Depressed=='Low BDI'],x='decisionCPE',y='band_pow',scatter=True,
           ci=95,color = '#824973',label='Low BDI',x_jitter = 0.15,scatter_kws={'s':5})
sns.regplot(data=reg_df[reg_df.Depressed=='High BDI'],x='decisionCPE',y='band_pow',scatter=True,
           ci=95,color='#0b4e5f',label='High BDI',x_jitter = 0.15,scatter_kws={'s':5})

plt.legend(fontsize=10)
# borderaxespad=0,frameon=False,framealpha=0,
sns.move_legend(ax,"upper center", bbox_to_anchor=(0.5, 1.1), frameon=False,framealpha=0,
                fontsize='small',shadow=None,
                columnspacing=2,ncol=2)


plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 
plt.tight_layout()
plt.ylabel(fr'${band}_{{{zpow}}}$')
plt.xlabel(r'$cpe(z)$')
sns.despine()
plt.tick_params(length=0)
# plt.text(7.25,0.07,f'rho: {np.round(r,3)} \n p: {np.round(p_r,6)}',fontsize=16)


# plt.savefig(f'{fig_save_dir}/all_lmplot_cpe_bpow_bdi_{date}.pdf', format='pdf', metadata=None,
# bbox_inches='tight', pad_inches=0.1,dpi=500,
# facecolor='auto', edgecolor='auto',
# backend=None)

In [ ]:
reg_df['Depressed'] = ['High BDI' if bdi >=20 else 'Low BDI' for bdi in reg_df.bdi.tolist()]

In [ ]:
## X plot attempt 

sns.set_theme(style='ticks') 
sns.set_context("poster")

# from matplotlib import rc
# rc('text', usetex=False)


fig,ax = plt.subplots(figsize=(8,5))
# sns.set_context("poster",rc={'axes.linewidth': 2})
# sns.set_theme(style='ticks') 

# palette={'depression':"#3f2755",'no depression':"#054a6f"}

# palette={'high BDI':"#0b4e5f",'low BDI':"#824973"}


# ax = sns.lmplot(data=reg_df,x='decisionCPE',y='band_pow',hue='Depression',scatter=False,ci=95,palette=palette)
# sns.move_legend(ax, loc='lower center', bbox_to_anchor=(0.1, 1),bbox_to_anchor=(-1, 0), ncol=2)

# sns.move_legend(ax,"upper center", bbox_to_anchor=(0.5, 1), borderaxespad=0.1,frameon=False,framealpha=0,
#                 fontsize='x-small',shadow=None,title=None,columnspacing=0.5,edgecolor=None,markerfirst=True,
#                 markerscale=4,labelcolor=None, ncol=2)


sns.regplot(data=reg_df[reg_df.Depressed=='Low BDI'],x='decisionCPE',y='band_pow',scatter=False,
           ci=95,color = '#824973',label='Low BDI')
sns.regplot(data=reg_df[reg_df.Depressed=='High BDI'],x='decisionCPE',y='band_pow',scatter=False,
           ci=95,color='#0b4e5f',label='High BDI')

plt.legend(fontsize=10)
# borderaxespad=0,frameon=False,framealpha=0,
sns.move_legend(ax,"upper center", bbox_to_anchor=(0.5, 1.1), frameon=False,framealpha=0,
                fontsize='small',shadow=None,
                columnspacing=2,ncol=2)


plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 
plt.tight_layout()
plt.ylabel(fr'${band}_{{{zpow}}}$')
plt.xlabel(r'$cpe(z)$')
sns.despine()
plt.tick_params(length=0)
# plt.text(7.25,0.07,f'rho: {np.round(r,3)} \n p: {np.round(p_r,6)}',fontsize=16)


# plt.savefig(f'{fig_save_dir}/all_lmplot_cpe_bpow_bdi_{date}.pdf', format='pdf', metadata=None,
# bbox_inches='tight', pad_inches=0.1,dpi=500,
# facecolor='auto', edgecolor='auto',
# backend=None)

In [ ]:
for roi in reg_df.roi.unique().tolist():
    roi_reg_plot = reg_df[reg_df.roi== roi]
    fig,ax = plt.subplots(figsize=(8,5))

    sns.regplot(data=roi_reg_plot[roi_reg_plot.Depressed=='Low BDI'],x='decisionCPE',y='band_pow',scatter=False,
               ci=95,color = '#824973',label='Low BDI')#,scatter_kws={'s':2})
    sns.regplot(data=roi_reg_plot[roi_reg_plot.Depressed=='High BDI'],x='decisionCPE',y='band_pow',scatter=False,
               ci=95,color='#0b4e5f',label='High BDI')#,scatter_kws={'s':2})

    plt.legend(fontsize=10)
    # borderaxespad=0,frameon=False,framealpha=0,
    sns.move_legend(ax,"upper center", bbox_to_anchor=(0.5, 1.1), frameon=False,framealpha=0,
                    fontsize='small',shadow=None,
                    columnspacing=2,ncol=2)


    plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 
    plt.tight_layout()
    plt.ylabel(fr'${band}_{{{zpow}}}$')
    plt.xlabel(r'$cpe(z)$')
    plt.title(f'{roi}')
    sns.despine()
    plt.tick_params(length=0)
    
    

In [ ]:
for roi in reg_df.roi.unique().tolist():
    roi_reg_plot = reg_df[reg_df.roi== roi]
    fig,ax = plt.subplots(figsize=(8,5))

    sns.regplot(data=roi_reg_plot[roi_reg_plot.Depressed=='Low BDI'],x='decisionCPE',y='band_pow',scatter=True,
               ci=95,color = '#824973',label='Low BDI',x_jitter = 0.15,scatter_kws={'s':5})
    sns.regplot(data=roi_reg_plot[roi_reg_plot.Depressed=='High BDI'],x='decisionCPE',y='band_pow',scatter=True,
               ci=95,color='#0b4e5f',label='High BDI',x_jitter = 0.15,scatter_kws={'s':5})

    plt.legend(fontsize=10)
    # borderaxespad=0,frameon=False,framealpha=0,
    sns.move_legend(ax,"upper center", bbox_to_anchor=(0.5, 1.1), frameon=False,framealpha=0,
                    fontsize='small',shadow=None,
                    columnspacing=2,ncol=2)


    plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 
    plt.tight_layout()
    plt.ylabel(fr'${band}_{{{zpow}}}$')
    plt.xlabel(r'$cpe(z)$')
    plt.title(f'{roi}')
    sns.despine()
    plt.tick_params(length=0)
    
    

In [ ]:
# for roi in reg_df.roi.unique().tolist():
#     roi_reg_plot = reg_df[reg_df.roi== roi]
#     fig,ax = plt.subplots(figsize=(8,5),dpi=500)

#     sns.regplot(data=roi_reg_plot[roi_reg_plot.Depressed=='Low BDI'],x='decisionCPE',y='band_pow',scatter=True,
#                ci=95,color = '#824973',label='Low BDI',x_jitter = 0.15)#,scatter_kws={'s':2})
#     sns.regplot(data=roi_reg_plot[roi_reg_plot.Depressed=='High BDI'],x='decisionCPE',y='band_pow',scatter=True,
#                ci=95,color='#0b4e5f',label='High BDI',x_jitter = 0.15)#,scatter_kws={'s':2})

#     plt.legend(fontsize=10)
#     # borderaxespad=0,frameon=False,framealpha=0,
#     sns.move_legend(ax,"upper center", bbox_to_anchor=(0.5, 1.1), frameon=False,framealpha=0,
#                     fontsize='small',shadow=None,
#                     columnspacing=2,ncol=2)


#     plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 

#     plt.ylabel(r'$beta_{{{zpow}}}$')
#     plt.xlabel(r'$cpe(z)$')
#     plt.title(f'{roi}')
#     sns.despine()
#     plt.tick_params(length=0)
    
    

In [ ]:
fig,ax = plt.subplots(figsize=(8,5),dpi=500)
# sns.set_context("poster",rc={'axes.linewidth': 2})
# sns.set_theme(style='ticks') 

# palette={'depression':"#3f2755",'no depression':"#054a6f"}

# palette={'high BDI':"#0b4e5f",'low BDI':"#824973"}


# ax = sns.lmplot(data=reg_df,x='decisionCPE',y='band_pow',hue='Depression',scatter=False,ci=95,palette=palette)
# sns.move_legend(ax, loc='lower center', bbox_to_anchor=(0.1, 1),bbox_to_anchor=(-1, 0), ncol=2)

# sns.move_legend(ax,"upper center", bbox_to_anchor=(0.5, 1), borderaxespad=0.1,frameon=False,framealpha=0,
#                 fontsize='x-small',shadow=None,title=None,columnspacing=0.5,edgecolor=None,markerfirst=True,
#                 markerscale=4,labelcolor=None, ncol=2)


sns.regplot(data=reg_df[reg_df.Depressed=='Low BDI'],x='decisionCPE',y='band_pow',scatter=True,
           ci=95,color = '#824973',label='Low BDI',x_jitter = 0.15,scatter_kws={'s':2})
sns.regplot(data=reg_df[reg_df.Depressed=='High BDI'],x='decisionCPE',y='band_pow',scatter=True,
           ci=95,color='#0b4e5f',label='High BDI',x_jitter = 0.15,scatter_kws={'s':2})

plt.legend(fontsize=10)
# borderaxespad=0,frameon=False,framealpha=0,
sns.move_legend(ax,"upper center", bbox_to_anchor=(0.5, 1.1), frameon=False,framealpha=0,
                fontsize='small',shadow=None,
                columnspacing=2,ncol=2)


plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 

plt.ylabel(fr'${band}_{{{zpow}}}$')
plt.xlabel(r'$cpe(z)$')
sns.despine()
plt.tick_params(length=0)
# plt.text(7.25,0.07,f'rho: {np.round(r,3)} \n p: {np.round(p_r,6)}',fontsize=16)



In [ ]:
palette=dict(Low="#3f2755", High="#054a6f")
# sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1), ncol=3, title=None, frameon=False)


In [ ]:
from scipy import stats
# slope, intercept, r_value, p_value, std_err = stats.linregress(df.x,df.y)

In [ ]:
reg_df['Depression'] = ['High BDI' if bdi >=20 else 'Low BDI' for bdi in reg_df.bdi.tolist()]

In [ ]:
# edgecolor=None,markerfirst=True,title=None,#title_fontsize='x-small',
#                 markerscale=0.2,labelcolor=None

In [ ]:
roi_order = ['dlpfc','vlpfc','ofc','ains','pins','acc','hpc','amy']

In [ ]:
lmplot

In [ ]:
reg_df

In [ ]:
lmplot = reg_df.copy()
lmplot = lmplot.groupby('unique_reref_ch').agg('mean').reset_index()

# beta_lmplot['Depressed'] = ['Low BDI' if bdi =< 20 else 'High BDI' for bdi in beta_lmplot.bdi]


# beta_lmplot['raw_rt_coeff']  = [roi_bdi_plot_data[roi_bdi_plot_data.unique_reref_ch == elec][
#     'raw_decisionCPE:band_pow'].values[0] for elec in beta_lmplot.unique_reref_ch]

# beta_lmplot['elec_bCPE']  = [roi_bdi_plot_data[roi_bdi_plot_data.unique_reref_ch == elec][
#     'elec_bCPE'].values[0] for elec in beta_lmplot.unique_reref_ch]

lmplot['roi']  = [reg_df[reg_df.unique_reref_ch == elec][
    'roi'].values[0] for elec in reg_df.unique_reref_ch]

lmplot = lmplot[lmplot.roi.isin(roi_order)]

lmplot

In [ ]:
roi_not_ains = ['notIns' if roi != 'ains' else 'ains' for roi in reg_df.roi.tolist()]
roi_not_ains

In [ ]:
reg_df['roi_not_ains'] = roi_not_ains

In [ ]:
final_vars = ['C(GambleChoice)', 'TotalProfit', 'decisionCPE']


In [ ]:

final_fit = fit_mixed_model(reg_df,final_vars,outcome_var,rand_eff_var,reml=False)
print(final_fit.summary())
    

In [ ]:
final_fit.pvalues['decisionCPE']

In [ ]:
mixed_effects_ftest_ttest(final_fit)

In [ ]:
fe_param_names = list(final_fit.fe_params.index[1:])
fe_param_names

In [ ]:
plot_data = pd.concat([pd.DataFrame(
    {**{'subj_id':elec_id.split('_')[0],'reref_ch_names':elec_id.split('_')[1]},'unique_reref_ch':elec_id,
     **{f'raw_{param}':final_fit.random_effects[elec_id][param] for param in list(
            final_fit.random_effects[elec_id].index)}},index=[range(len(fe_param_names))])
        
    for elec_id in final_fit.random_effects.keys()]).reset_index(drop=True)

plot_data['roi'] = [reg_df.roi[reg_df.unique_reref_ch == row.unique_reref_ch].values[0]
                                for idx,row in plot_data.iterrows()]

plot_data['bdi'] = [subj_info_df.bdi[subj_info_df.subj_id == subj_id].values[0]
                                for subj_id in plot_data.subj_id]

In [ ]:
plot_data

In [ ]:
#### Get all results for Figs!!!!

fe_intercept   = final_fit.params['Intercept']
fe_cpe_beta    = final_fit.params['decisionCPE']
fe_cpe_beta


In [ ]:
#### add fixed effects data to all_elec_plot_data
plot_data['elec_b0']   = np.add(plot_data.raw_Group,fe_intercept)
plot_data['elec_bCPE'] = np.add(plot_data.raw_decisionCPE,fe_cpe_beta)
plot_data = plot_data[~plot_data.roi.isnull()]

In [ ]:
plot_data

In [ ]:
raw_roi_stats = {f'{roi}':ttest_1samp(plot_data.raw_decisionCPE[plot_data.roi == roi].unique(),
                 popmean=0) for roi in plot_data.roi.unique().tolist()}
raw_roi_stats

In [ ]:
roi_stats = {f'{roi}':ttest_1samp(plot_data.elec_bCPE[plot_data.roi == roi].unique(),
                 popmean=0) for roi in plot_data.roi.unique().tolist()}
roi_stats

In [ ]:
bonferr_alpha = 0.05/(len(plot_data.roi.unique().tolist()))
bonferr_alpha

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,raw_roi_stats[roi].pvalue) for roi in raw_roi_stats.keys()
                                                   if raw_roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())


In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,roi_stats[roi].pvalue) for roi in roi_stats.keys()
                                                   if roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())




In [ ]:
roi_stats.keys()

In [ ]:
# roi_order = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy','mtg','motor','thalamus']
roi_order = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy','mtg']

In [ ]:
roi_plot_data = plot_data.copy()
roi_plot_data = roi_plot_data[roi_plot_data.roi.isin(roi_order)]

# roi_plot_names = {'thalamus':'THAL','hpc':'HPC','amy':'AMY','acc':'ACC','ofc':'OFC','ains':'aINS',
#                   'pins':'pINS','dmpfc':'dmPFC','dlpfc':'dlPFC','vlpfc':'vlPFC'}


In [ ]:
# roi_plot_data

In [ ]:
sns.set_context("poster",rc={'axes.linewidth': 1.5})
fig,ax = plt.subplots(figsize=(15, 6),dpi=500)

fig_save_dir = f'{base_dir}ephys_analysis/figs/{band}_regressions/'
os.makedirs(fig_save_dir,exist_ok=True)

sns.barplot(data = roi_plot_data,
            x = 'roi',
#             y = 'elec_bCPE',
            y='raw_decisionCPE',
#             color = '#561C46',
#             color = '#00467e',
#             color = '#346a97',
            color = '#346a97',
            saturation=1,
            order=roi_order,
            errorbar=None) 


sns.stripplot(data = roi_plot_data,
            x = 'roi',
#             y = 'elec_bCPE',
              y='raw_decisionCPE',
#               color='#78496b', 
              color = '#00467e',
              alpha=.4,
              dodge=True,
              order=roi_order,
              size = 8)

ax.axhline(y=0,color='#00467e',linewidth=1.5)
plt.tick_params(length=0 )#,grid_linewidth=1)

# plt.ylim([-0.05,0.05])
ax.set(xlabel=None)
# plt.ylabel('CPE Encoding in Beta Power (z)')
# plt.ylabel('beta(z) cpe encoding')
plt.ylabel(fr'${band}_{{{zpow}}}$'+ ' cpe encoding')
# plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}')) 
# plt.locator_params(axis='y', nbins=5,tight=True)


sns.despine()
    
plt.savefig(f'{fig_save_dir}/Bpow_CPE_barplot_{date}.pdf', format='pdf', metadata=None,
bbox_inches=None, pad_inches=0.1,dpi=500,
facecolor='auto', edgecolor='auto',
backend=None)


In [ ]:
rho_data = []
sig_rho = []

# BDI CORRELATIONS
for roi in plot_data.roi.unique().tolist()[:-1]:
    roi_temp_data = plot_data[plot_data.roi==roi]
    rho,rho_p     = pearsonr(roi_temp_data.bdi,roi_temp_data.raw_decisionCPE)
#     rho,rho_p     = pearsonr(roi_temp_data.bdi,roi_temp_data.elec_bCPE)
#     print(f'{roi} pearsonr',rho,rho_p)
    rho_data.append(pd.DataFrame({'roi':roi,'rho':rho,'rho_p':rho_p},index=[0]))
#     if rho_p <= bonferr_alpha:
#         sig_bdi_rois.append(roi)
    if rho_p < bonferr_alpha: 
        sig_rho.append((roi,rho,rho_p))
    print(f'{roi} pearsonr',rho,rho_p)
rho_data = pd.concat(rho_data).reset_index(drop=True)


In [ ]:
sig_rho

In [ ]:
rho_data['plot_p'] = [0.001 if roi.rho_p < 0.001 else np.round(roi.rho_p,4) for ix,roi in rho_data.iterrows()]

In [ ]:
new_plot_ps = []
for p in rho_data['plot_p']:
    new_p = 0
    
    for ix,val in enumerate(str(p)):
        if val == '0':
            continue
        elif val == '.':
            continue
        else: 
#             print(ix,val)
            new_p = str(p)[:ix+1]
            break
            
            

    new_plot_ps.append(float(new_p))
#         print(val)

In [ ]:
rho_data['round_p']   = new_plot_ps
rho_data['round_rho'] = np.round(rho_data.rho,2)
rho_data

In [ ]:

sns.set_context("poster",rc={'axes.linewidth': 1.5})

for roi in plot_data.roi.unique().tolist()[:-1]:
    roi_temp_data = plot_data[plot_data.roi==roi]

    rho   = rho_data[rho_data.roi == roi].round_rho.values[0]
    rho_p = rho_data[rho_data.roi == roi].round_p.values[0]
    
        
    fig = plt.figure(figsize=(7,5),dpi=300)

    sns.regplot(data=roi_temp_data,x='bdi',y='raw_decisionCPE',scatter=False,ci=95,
                fit_reg=True,color = '#00467e') #034975
    
    sns.scatterplot(data = roi_temp_data.groupby(['subj_id']).agg('mean'),x='bdi',
                    y='raw_decisionCPE',color = '#00467e',edgecolor=None,alpha=0.8)
    
#     plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}')) 
    plt.xlabel('BDI-II')
    
    plt.tick_params(length=0) 
    plt.ylabel(band+r'$_{{{zpow}}}$'+' cpe encoding')

    plt.text(np.max(roi_temp_data.bdi)-8,
             np.round(np.max(roi_temp_data.groupby(['subj_id']).agg('mean').raw_decisionCPE),3)-0.003,
             r'$rho$ = '+f'{rho} '+'\n '+r'$p < $'+f'{rho_p}',
             color='#00467e',fontsize=16)

    plt.title(f'{roi}',fontsize=24)


    
    sns.despine()
    
    plt.savefig(f'{fig_save_dir}/{roi}_{rho}_{rho_p}_roi_bdi_{band}-cpe-slope.pdf', format='pdf', metadata=None,
        bbox_inches='tight', pad_inches=0.1,dpi=300,
        facecolor='auto', edgecolor='auto',
        backend=None)
    

# BDI Interaction
- Rerun final mixed effects model on each ROI separately
- Correlate random effects slopes for each regressor with BDI

In [ ]:
rand_eff_var = 'unique_reref_ch'


In [ ]:
reg_df

In [ ]:
bdi_vars    = ['decisionCPE','bdi','decisionCPE:bdi']
outcome_var = 'band_pow'

bdi_fit = fit_mixed_model(reg_df,bdi_vars,outcome_var,rand_eff_var,reml=False)
print(bdi_fit.summary())
    

In [ ]:
fe_param_names = list(bdi_fit.fe_params.index[1:])
fe_param_names

In [ ]:
bdi_plot_data = pd.concat([pd.DataFrame(
    {**{'subj_id':elec_id.split('_')[0],'reref_ch_names':elec_id.split('_')[1]},'unique_reref_ch':elec_id,
     **{f'raw_{param}':bdi_fit.random_effects[elec_id][param] for param in list(
            bdi_fit.random_effects[elec_id].index)}},index=[range(len(fe_param_names))])
        
    for elec_id in bdi_fit.random_effects.keys()]).reset_index(drop=True)

bdi_plot_data['roi'] = [reg_df.roi[reg_df.unique_reref_ch == row.unique_reref_ch].values[0]
                                for idx,row in bdi_plot_data.iterrows()]

bdi_plot_data['bdi'] = [subj_info_df.bdi[subj_info_df.subj_id == subj_id].values[0]
                                for subj_id in bdi_plot_data.subj_id]

In [ ]:
bdi_plot_data

In [ ]:
#### Get all results for Figs!!!!

fe_intercept   = bdi_fit.params['Intercept']
fe_cpe_beta    = bdi_fit.params['decisionCPE:bdi']
fe_cpe_beta


In [ ]:
#### add fixed effects data to all_elec_plot_data
bdi_plot_data['elec_b0']   = np.add(bdi_plot_data.raw_Group,fe_intercept)
bdi_plot_data['elec_bCPE'] = np.add(bdi_plot_data['raw_decisionCPE:bdi'],fe_cpe_beta)
bdi_plot_data = bdi_plot_data[~bdi_plot_data.roi.isnull()]

In [ ]:
bdi_plot_data

In [ ]:
# raw_bdi

raw_roi_stats = {f'{roi}':ttest_1samp(bdi_plot_data['raw_bdi'][bdi_plot_data.roi == roi].unique(),
                 popmean=0) for roi in bdi_plot_data.roi.unique().tolist()}
raw_roi_stats

In [ ]:
raw_roi_stats = {f'{roi}':ttest_1samp(bdi_plot_data['raw_decisionCPE:bdi'][bdi_plot_data.roi == roi].unique(),
                 popmean=0) for roi in bdi_plot_data.roi.unique().tolist()}
raw_roi_stats

In [ ]:
roi_stats = {f'{roi}':ttest_1samp(bdi_plot_data.elec_bCPE[bdi_plot_data.roi == roi].unique(),
                 popmean=0) for roi in bdi_plot_data.roi.unique().tolist()}
roi_stats

In [ ]:
bonferr_alpha = 0.05/len(roi_stats)
# bonferr_alpha = 0.05/len(bdi_plot_data.roi.unique().tolist())
bonferr_alpha

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,raw_roi_stats[roi].pvalue) for roi in raw_roi_stats.keys()
                                                   if raw_roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())


In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,roi_stats[roi].pvalue) for roi in roi_stats.keys()
                                                   if roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())




In [ ]:
roi_stats

In [ ]:
roi_stats.keys()

In [ ]:
# roi_order = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy','mtg','motor','thalamus']
roi_order = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy']

In [ ]:
roi_bdi_plot_data = bdi_plot_data.copy()
roi_bdi_plot_data = roi_bdi_plot_data[roi_bdi_plot_data.roi.isin(roi_order)]

# roi_plot_names = {'thalamus':'THAL','hpc':'HPC','amy':'AMY','acc':'ACC','ofc':'OFC','ains':'aINS',
#                   'pins':'pINS','dmpfc':'dmPFC','dlpfc':'dlPFC','vlpfc':'vlPFC'}


In [ ]:
roi_bdi_plot_data['raw_decisionCPE:bdi']

In [ ]:
sns.set_context("poster")
fig,ax = plt.subplots(figsize=(15, 6),dpi=500)

fig_save_dir = f'{base_dir}ephys_analysis/figs/{band}_regressions/'
os.makedirs(fig_save_dir,exist_ok=True)

sns.barplot(data = roi_bdi_plot_data,
            x = 'roi',
            y = 'raw_decisionCPE:bdi',
#             y='elec_bCPE',
            color = '#561C46', #173b6f
            saturation=0.8,
            order=roi_order,
            errorbar=None) 


sns.stripplot(data = roi_bdi_plot_data,
            x = 'roi',
            y = 'raw_decisionCPE:bdi',
              color = '#561C46',
              alpha=.4,
              dodge=True,
              order=roi_order,
              size = 8)

ax.axhline(y=0,color='#561C46',linewidth=1.5)
plt.tick_params(length=0 )#,grid_linewidth=1)

# plt.ylim([-0.02,0.017])
ax.set(xlabel=None)

plt.ylabel(fr'${band}_{{{zpow}}} \sim cpe:bdi$')
# plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}')) 
plt.locator_params(axis='y', nbins=6,tight=True)



sns.despine()
    
plt.savefig(f'{fig_save_dir}/{band}pow_CPEBDIINT{date}.pdf', format='pdf', metadata=None,
bbox_inches=None, pad_inches=0.1,dpi=500,
facecolor='auto', edgecolor='auto',
backend=None)

In [ ]:
####### BDI ~ BPOW SIG FOR THESE ROIS!!!

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,raw_roi_stats[roi].pvalue) for roi in raw_roi_stats.keys()
                                                   if raw_roi_stats[roi].pvalue< bonferr_alpha])
# print('sig ROI pvals: ',roi_stats.roi_cpe_pval[roi_beta_results.roi_cpe_pval<=bonferr_alpha].unique().tolist())


In [ ]:
roi_bdi_plot_data

In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,ttest_1samp(roi_bdi_plot_data[
    roi_bdi_plot_data.roi == roi]['raw_decisionCPE:bdi'].to_numpy(),
                                     popmean=0).pvalue) for roi in roi_bdi_plot_data.roi.unique().tolist()
                                                  if ttest_1samp(roi_bdi_plot_data[
    roi_bdi_plot_data.roi == roi]['raw_decisionCPE:bdi'].to_numpy(),
                                     popmean=0).pvalue < bonferr_alpha])




In [ ]:
print('sig ROIs after multiple comp correction: ',[(roi,ttest_1samp(roi_bdi_plot_data[
    roi_bdi_plot_data.roi == roi]['raw_bdi'].to_numpy(),
                                     popmean=0).pvalue) for roi in roi_bdi_plot_data.roi.unique().tolist()
                                                  if ttest_1samp(roi_bdi_plot_data[
    roi_bdi_plot_data.roi == roi]['raw_bdi'].to_numpy(),
                                     popmean=0).pvalue < bonferr_alpha])




In [ ]:
sns.set_context("poster")
fig,ax = plt.subplots(figsize=(15, 6),dpi=500)

fig_save_dir = f'{base_dir}ephys_analysis/figs/{band}_regressions/'
os.makedirs(fig_save_dir,exist_ok=True)

sns.barplot(data = roi_bdi_plot_data,
            x = 'roi',
            y = 'raw_bdi',
#             y='elec_bCPE',
            color = '#173b6f', #173b6f
            saturation=0.8,
            order=roi_order,
            errorbar=None) 


sns.stripplot(data = roi_bdi_plot_data,
            x = 'roi',
            y = 'raw_bdi',
              color = '#173b6f',
              alpha=.4,
              dodge=True,
              order=roi_order,
              size = 8)

ax.axhline(y=0,color='#173b6f',linewidth=1.5)
plt.tick_params(length=0 )#,grid_linewidth=1)

# plt.ylim([-0.02,0.017])
ax.set(xlabel=None)

# plt.ylabel(r'$beta_{{{zpow}}} \sim cpe:bdi$')
plt.ylabel(fr'${band}_{{{zpow}}} \sim bdi$')
# plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}')) 
plt.locator_params(axis='y', nbins=6,tight=True)



sns.despine()
    
plt.savefig(f'{fig_save_dir}/{band}POW~BDIbyROI{date}.pdf', format='pdf', metadata=None,
bbox_inches=None, pad_inches=0.1,dpi=500,
facecolor='auto', edgecolor='auto',
backend=None)

In [ ]:
roi_bdi_plot_data

In [ ]:
# roi_order = ['dlpfc','vlpfc','dmpfc','ofc','ains','pins','acc','hpc','amy']

In [ ]:
reg_df.groupby('roi')

In [ ]:
reg_df['BDI-II'] = ['Low' if bdi<20 else 'High' for bdi in reg_df.bdi]
reg_df

In [ ]:
palette=dict(Low="#3f2755", High="#054a6f")
# sns.move_legend(ax, "lower center", bbox_to_anchor=(.5, 1), ncol=3, title=None, frameon=False)


In [ ]:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(df.x,df.y)

In [ ]:
# reg_df['Depression'] = ['depressed' if bdi >=20 else 'not depressed' for bdi in reg_df.bdi.tolist()]
reg_df['Depression'] = ['high BDI' if bdi >=20 else 'low BDI' for bdi in reg_df.bdi.tolist()]

In [ ]:
# edgecolor=None,markerfirst=True,title=None,#title_fontsize='x-small',
#                 markerscale=0.2,labelcolor=None

In [ ]:
reg_df.roi.unique().tolist()

# INTERACTION FEATURE SELECTION

In [ ]:
######## make all possible variable combinations for regressions 
main_var  = ['decisionCPE','band_pow','decisionCPE:band_pow']

### iter vars for RT prediction dfs > 
iter_vars = ['C(GambleChoice)','TrialEV','TotalProfit',
             'C(GambleChoice_t1)','TrialEV_t1']


In [ ]:
iter_vars

In [ ]:
var_combos = [[v]+main_var for v in iter_vars]

for i in range(1,len(iter_vars)-1):
    iter_list = [list(tup)+main_var for tup in list(itertools.combinations(iter_vars,i+1))]
    var_combos.extend(iter_list)

# remove var combos that contain Choice + Trial Type for the *same trial* only bc they are collinear
var_combos = [combo for combo in var_combos if ('C(GambleChoice_t1)' in combo and 'C(TrialType_t1)' not in combo)|
              ('C(GambleChoice_t1)' not in combo and 'C(TrialType_t1)' in combo)|('C(GambleChoice_t1)'not in combo and 'C(TrialType_t1)' not in combo)]

var_combos = [combo for combo in var_combos if ('C(GambleChoice)' in combo and 'C(TrialType)' not in combo)|
              ('C(GambleChoice)' not in combo and 'C(TrialType)' in combo)|('C(GambleChoice)'not in combo and 'C(TrialType)' not in combo)]


var_combos = [combo for combo in var_combos if ('C(GambleChoice)' in combo and 'TrialEV' not in combo)|
              ('C(GambleChoice)' not in combo and 'TrialEV' in combo)|('C(GambleChoice)'not in combo and 'TrialEV' not in combo)]


var_combos = [combo for combo in var_combos if ('TrialEV' in combo and 'C(TrialType)' not in combo)|
              ('TrialEV' not in combo and 'C(TrialType)' in combo)|('TrialEV'not in combo and 'C(TrialType)' not in combo)]

var_combos = [combo for combo in var_combos if ('C(GambleChoice_t1)' in combo and 'TrialEV_t1' not in combo)|
              ('C(GambleChoice_t1)' not in combo and 'TrialEV_t1' in combo)|('C(GambleChoice_t1)'not in combo and 'TrialEV_t1' not in combo)]

var_combos = [combo for combo in var_combos if ('TrialEV_t1' in combo and 'C(TrialType_t1)' not in combo)|
              ('TrialEV_t1' not in combo and 'C(TrialType_t1)' in combo)|('TrialEV_t1'not in combo and 'C(TrialType_t1)' not in combo)]
var_combos

In [ ]:
len(var_combos)

In [ ]:
# import time 
start = time.time() # start timer

all_models = {}
all_rsq    = {}
all_stats  = {}
all_keys   = {}
no_conv    = []

for test_vars in var_combos:
    model_key = ''.join(test_vars[:-1]) ### make string of covariate var names to use as dict key
    model_fit = fit_mixed_model(reg_df,test_vars,outcome_var,rand_eff_var,reml=False)
    # check if model converged
    if model_fit.converged:
        
        print(model_key,' converged!')
        
        all_models[model_key] = model_fit
        all_rsq[model_key]    = compute_marginal_rsq(model_fit)
        all_stats[model_key]  = mixed_effects_ftest_ttest(model_fit)
        all_keys[model_key]   = test_vars
        
    else:
        print(model_key,' did not converge :(')
        no_conv.append(model_key)
    del model_fit

end = time.time()
print('{:.4f} s'.format(end-start)) # print time elapsed for computation (approx 20 seconds per channel)    

In [ ]:
no_conv

In [ ]:
list(all_models.keys())

In [ ]:
all_models['TrialEVdecisionCPEband_pow'].bic

In [ ]:
model_summary = {key:all_models[key].bic for key in all_models.keys()}
model_summary

In [ ]:
bic_list = [model_summary[key] for key in model_summary.keys()]
bic_list.sort()

ordered_list = [list(model_summary.keys())[np.where(list(model_summary.values()) == bic)[0][0]] for bic in bic_list]
list(zip(ordered_list,bic_list))


In [ ]:
all_models['TotalProfitTrialEV_t1decisionCPEband_pow'].pvalues

In [ ]:
[(key,bic,all_models[key].pvalues['decisionCPE:band_pow']) for key,bic in list(zip(ordered_list,bic_list))]  


In [ ]:
print('worst bic model: ',list(model_summary.keys())[np.where(list(model_summary.values())==np.max(list(model_summary.values())))[0][0]])


In [ ]:
print('best bic model: ',list(model_summary.keys())[np.where(list(model_summary.values())==np.min(list(model_summary.values())))[0][0]])


In [ ]:
print(list(model_summary.keys())[np.where(list(model_summary.values())==np.min(list(model_summary.values())))[0][0]])
print(np.min(list(model_summary.values())))
      
      

In [ ]:
[(key,all_models[key].pvalues['decisionCPE:band_pow']) for key in ordered_list if all_models[key].pvalues['decisionCPE:band_pow'] < 0.05]  



In [ ]:
[(key,all_models[key].pvalues['decisionCPE:band_pow']) for key in ordered_list if all_models[key].pvalues['decisionCPE:band_pow'] > 0.05]  


In [ ]:
all_models['TrialEVTotalProfitdecisionCPEband_pow'].pvalues

In [ ]:
all_models['TrialEVTotalProfitdecisionCPEband_pow'].params

In [ ]:
all_models['TotalProfitTrialEV_t1decisionCPEband_pow'].summary()

In [ ]:
all_rsq

In [ ]:
all_models

In [ ]:
all_stats['TrialEVdecisionCPEband_pow']

In [ ]:
all_stats

In [ ]:
#### FOR NOW JUST SAVE CPE STATS!
t_test_results_summary = [pd.DataFrame({'ttest_fe_tval':stats[0].t.tolist()[-1],
                                        'ttest_fe_coef' :stats[0].coef.tolist()[-1],
                                        'ttest_fe_pval':stats[0]['P>|t|'].tolist()[-1],
                                        'model_id':idx},index=[0]) 
                          for idx,stats in all_stats.items()]


t_test_results_summary = pd.concat(t_test_results_summary).reset_index(drop=True)
t_test_results_summary



In [ ]:
f_test_results_summary = [pd.DataFrame({'ftest_fvalue':stats[1].fvalue,
                                        'ftest_re_df':stats[1].df_num,
                                        'ftest_f_pval':stats[1].pvalue,
                                        'model_id':idx},index=[0]) 
                          for idx,stats in all_stats.items()]

f_test_results_summary = pd.concat(f_test_results_summary).reset_index(drop=True)
f_test_results_summary

In [ ]:
tstat_fstat_df = pd.merge(t_test_results_summary,f_test_results_summary,left_on=['model_id'],
                          right_on=['model_id']).reset_index(drop=True)

tstat_fstat_df = tstat_fstat_df[['model_id','ttest_fe_tval', 'ttest_fe_coef', 'ttest_fe_pval',
                                 'ftest_fvalue', 'ftest_re_df', 'ftest_f_pval']]
tstat_fstat_df



In [ ]:
tstat_fstat_df['rsq'] = [val for key,val in all_rsq.items()]

In [ ]:
tstat_fstat_df

In [ ]:
all_rsq

In [ ]:

tstat_fstat_df['bic']         = [all_models[key].bic for key in all_models.keys()]


In [ ]:
tstat_fstat_df

In [ ]:
tstat_fstat_df['fe_cpe_est'] = [all_models[key].params['decisionCPE:band_pow'] for key in all_models.keys()]

In [ ]:
tstat_fstat_df['fe_cpe_pval'] = [all_models[key].pvalues['decisionCPE:band_pow'] for key in all_models.keys()]

In [ ]:
tstat_fstat_df



In [ ]:
#### multiple comparisons correction!!!!
bonferr_alpha = 0.05/len(tstat_fstat_df)
bonferr_alpha

In [ ]:

tstat_fstat_df.sort_values(by='fe_cpe_pval')

In [ ]:
# tstat_fstat_df = tstat_fstat_df[tstat_fstat_df.fe_cpe_pval < bonferr_alpha]
tstat_fstat_df.sort_values(by='ttest_fe_pval')

In [ ]:
save_df = tstat_fstat_df.copy()

In [ ]:
tstat_fstat_df.sort_values(by='ttest_fe_pval')

In [ ]:
tstat_fstat_df.ttest_fe_pval

In [ ]:
print('Best BIC model: ',tstat_fstat_df.model_id[tstat_fstat_df.bic == np.min(tstat_fstat_df.bic)].values[0])


In [ ]:
print('Best fe_cpe_pval model: ',tstat_fstat_df.model_id[tstat_fstat_df.fe_cpe_pval == np.min(tstat_fstat_df.fe_cpe_pval)].values[0])


In [ ]:
print('Best rsq model: ',tstat_fstat_df.model_id[tstat_fstat_df.rsq == np.max(tstat_fstat_df.rsq)].values[0])


In [ ]:
print('Best ttest pval: ',tstat_fstat_df.model_id[tstat_fstat_df.ttest_fe_pval == 
                                                  np.min(tstat_fstat_df.ttest_fe_pval)].values[0])


In [ ]:
print('Best tstat: ',tstat_fstat_df.model_id[tstat_fstat_df.ttest_fe_tval == 
                                                  np.max(tstat_fstat_df.ttest_fe_tval)].values[0])


In [ ]:
print('Best ftest pval: ',tstat_fstat_df.model_id[tstat_fstat_df.ftest_f_pval == 
                                                  np.min(tstat_fstat_df.ftest_f_pval)].values[0])

In [ ]:
print('Best Fstat model: ',tstat_fstat_df.model_id[tstat_fstat_df.ftest_fvalue == 
                                                  np.max(tstat_fstat_df.ftest_fvalue)].values[0])

In [ ]:
all_models_fe_data   = {}

for key in all_models.keys():
#     model_unique_elecs = all_models[key].random_effects.keys()
    model_fe_params = {f'{param}':all_models[key].fe_params[param] for param in all_models[key].fe_params.index}
    model_fe_params['model_id'] = key
    all_models_fe_data[key] = model_fe_params



In [ ]:
all_models_fe_data

In [ ]:
model_plot_data = {}

for key in all_models.keys():
    fe_param_names = list(all_models[key].fe_params.index)
    
    plot_data = pd.concat([pd.DataFrame(
    {**{'subj_id':elec_id.split('_')[0],'reref_ch_names':elec_id.split('_')[1]},'unique_reref_ch':elec_id,
     **{f'raw_{param}':all_models[key].random_effects[elec_id][param] for param in list(
            all_models[key].random_effects[elec_id].index)}},index=[range(len(fe_param_names))])
    for elec_id in all_models[key].random_effects.keys()]).reset_index(drop=True)
    
    plot_data['roi'] = [reg_df.roi[reg_df.unique_reref_ch == row.unique_reref_ch].values[0]
                                for idx,row in plot_data.iterrows()]

    plot_data['bdi'] = [subj_info_df.bdi[subj_info_df.subj_id == subj_id].values[0]
                                for subj_id in plot_data.subj_id]
    model_plot_data[key] = plot_data

